#   <font color='blue'> WeRateDogs Twitter Data Wrangling </font> 

## Introduction

The dataset that I will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.


### Data wrangling process


The process takes place in 3 steps:

    * Gather 
    * Assess 
    * Clean 

We can assess data for:
    * Quality: issues with content. Low quality data is also known as dirty data.
    * Tidiness: issues with structure that prevent easy analysis. Untidy data is also known as messy data. 
Tidy data requirements:
    1.	Each variable forms a column.
    2.	Each observation forms a row.
    3.	Each type of observational unit forms a table.

...using two types of assessment:
    * Visual assessment: scrolling through the data in your preferred software application (Google Sheets, Excel, a text editor, etc.).
    * Programmatic assessment: using code to view specific portions and summaries of the data (pandas' head, tail, and info methods, for example).


Rules to follow when wrangling this data
      
    * We only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

    * The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.


##  Gather

In [87]:
# import pckages
import pandas as pd
import requests
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
% matplotlib inline

1. Load twitter-archive-enhanced csv file in which data extracted from @dog_rates user tweets are stored. The archive from which this data is extracted contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of its tweets as they stood on August 1, 2017.

In [2]:
tw = pd.read_csv('twitter-archive-enhanced.csv')

2. Using 'requests' package, download image-predictions tsv file, save it in the current dirrectory, then load it as a padas dataframe in the notebook.

In [3]:
filename = 'image-predictions.tsv'
url =  'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(filename, 'wb') as fd:
        fd.write(response.content)

In [3]:
im = pd.read_csv('image-predictions.tsv', sep=('\t'))

3. Gathering more data about @dog_rates user tweets than that stored in twitter-archive-enhanced.csv file. For This I will use the tweepy library.

In [5]:
# by this code, a tweeter API object is created. It will be used to gather tweeter data
import tweepy

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [6]:
# Get all tweet ids in the tw table.
tw_ids = tw.tweet_id
tw_ids.nunique()

2356

In [62]:
# Tweet data is stored in JSON format by Twitter.
# This code get the tweet JSON data via tweet ID 
  
with open('tweet_json.txt', 'w') as f:
    for tweet_id in tw_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode = 'extended') 
            json.dump(tweet._json, f)
            f.write('\n') # Retour à la ligne
        except:
            continue



Rate limit reached. Sleeping for: 85
Rate limit reached. Sleeping for: 729


In [4]:
#I will extract tweet id, retweet_count, favorite_count and truncated
# data from the extansion tweet data and I will stored them in a pandas dataframe. 

# retweet_count: Number of times this Tweet has been retweeted

# favorite_count: Indicates approximately how many times this Tweet has been liked by Twitter users. 


# truncated: Indicates whether the value of the text parameter was truncated, for example, 
# as a result of a retweet exceeding the original Tweet text length limit of 140 characters. 
# Truncated text will end in ellipsis, like this ... 
# Since Twitter now rejects long Tweets vs truncating them, 
# the large majority of Tweets will have this set to false . 
# Note that while native retweets may have their toplevel text property shortened, 
# the original text will be available under the retweeted_status object and 
# the truncated parameter will be set to the value of the original status (in most cases, false ).
# Perhaps I will use truncated variable to show more tweet text if needed.

tw_ext = pd.DataFrame(columns = {'tweet_id','retweet_count','favorite_count','truncated'} )

with open('tweet_json.txt', 'r') as f:
        for line in f:
            tweet = json.loads(line)
            tweet_id = tweet['id']
            retweet_count = tweet['retweet_count']
            favorite_count = tweet['favorite_count']
            truncated = tweet['truncated']
            tw_ext = tw_ext.append({'tweet_id':tweet_id, 'retweet_count':retweet_count,
                'favorite_count':favorite_count, 'truncated': truncated}, ignore_index=True)
            

In [39]:
tweet

{'created_at': 'Sun Nov 15 22:32:08 +0000 2015',
 'id': 666020888022790149,
 'id_str': '666020888022790149',
 'full_text': 'Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj',
 'truncated': False,
 'display_text_range': [0, 131],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 666020881337073664,
    'id_str': '666020881337073664',
    'indices': [108, 131],
    'media_url': 'http://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
    'url': 'https://t.co/BLDqew2Ijj',
    'display_url': 'pic.twitter.com/BLDqew2Ijj',
    'expanded_url': 'https://twitter.com/dog_rates/status/666020888022790149/photo/1',
    'type': 'photo',
    'sizes': {'medium': {'w': 960, 'h': 720, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'large': {'w': 960, 'h': 720, 'resize': 'fit

## Assess

In [6]:
tw.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [24]:
tw.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [28]:
tw[['rating_numerator','rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


In [35]:
sum(tw.duplicated())

0

In [68]:
tw.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [22]:
tw.rating_numerator.sort_values(), tw.rating_numerator.value_counts()

(315        0
 1016       0
 2335       1
 2261       1
 2338       1
 605        1
 1446       1
 1869       1
 2091       1
 2038       1
 1940       1
 1764       2
 2310       2
 1761       2
 1920       2
 2246       2
 2237       2
 2349       2
 2326       2
 2079       2
 2305       3
 1249       3
 1898       3
 1478       3
 1692       3
 2202       3
 765        3
 1928       3
 1314       3
 2183       3
         ... 
 866       14
 83        14
 291       15
 285       15
 55        17
 1663      20
 516       24
 1712      26
 763       27
 1433      44
 1274      45
 1202      50
 1351      60
 340       75
 695       75
 1254      80
 433       84
 1843      88
 1228      99
 1635     121
 1634     143
 1779     144
 902      165
 290      182
 1120     204
 2074     420
 188      420
 189      666
 313      960
 979     1776
 Name: rating_numerator, Length: 2356, dtype: int64, 12      558
 11      464
 10      461
 13      351
 9       158
 8       102
 7        55
 14

In [19]:
tw.rating_denominator.sort_values()[0:10], tw.rating_denominator.value_counts()[0:10]

(313      0
 2335     2
 516      7
 1576    10
 1575    10
 1574    10
 1573    10
 1572    10
 1563    10
 1570    10
 Name: rating_denominator, dtype: int64, 10    2333
 11       3
 50       3
 80       2
 20       2
 2        1
 16       1
 40       1
 70       1
 15       1
 Name: rating_denominator, dtype: int64)

In [111]:
tw.text[2074], tw.rating_denominator[2074]

('After so many requests... here you go.\n\nGood dogg. 420/10 https://t.co/yfAAo1gdeY',
 10)

In [96]:
tw.name.value_counts()

None              745
a                  55
Charlie            12
Lucy               11
Cooper             11
Oliver             11
Lola               10
Tucker             10
Penny              10
Winston             9
Bo                  9
the                 8
Sadie               8
Buddy               7
Bailey              7
an                  7
Toby                7
Daisy               7
Koda                6
Oscar               6
Rusty               6
Milo                6
Jack                6
Stanley             6
Leo                 6
Jax                 6
Dave                6
Bella               6
Scout               6
Louis               5
                 ... 
unacceptable        1
Hero                1
Arnold              1
Travis              1
Charleson           1
his                 1
Rumpole             1
Rambo               1
Dudley              1
Nugget              1
Mosby               1
Jiminus             1
Bertson             1
Asher               1
Miley     

In [9]:
tw.name.sort_values()

1035            Abby
1021            Abby
938              Ace
1933            Acro
1327           Adele
1934           Aiden
77               Aja
480            Akumi
820               Al
875           Albert
1954          Albert
144            Albus
412            Albus
1115         Aldrick
2046       Alejandro
374        Alexander
1189    Alexanderson
486              Alf
2238           Alfie
661            Alfie
367            Alfie
1616           Alfie
858            Alfie
51              Alfy
201            Alice
1701           Alice
1495           Amber
1334         Ambrose
2078             Amy
2195          Amélie
            ...     
852               my
988              not
335              not
1747      officially
1206             old
993              one
1936             one
924              one
369              one
2326           quite
169            quite
118            quite
193            quite
2030           space
22              such
2345             the
2346         

In [104]:
tw.doggo.value_counts() , tw.floofer.value_counts(), tw.pupper.value_counts(), tw.puppo.value_counts()

(None     2259
 doggo      97
 Name: doggo, dtype: int64, None       2346
 floofer      10
 Name: floofer, dtype: int64, None      2099
 pupper     257
 Name: pupper, dtype: int64, None     2326
 puppo      30
 Name: puppo, dtype: int64)

In [32]:
tw[(tw['doggo'] != 'None') & (tw['floofer'] != 'None')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/854010172...,11,10,None,doggo,floofer,None,None


In [33]:
tw[(tw['doggo'] != 'None') & (tw['pupper'] != 'None')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,doggo,None,pupper,None
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending it pictures that don't eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751583847...,5,10,None,doggo,None,pupper,None


In [34]:
tw[(tw['doggo'] != 'None') & (tw['puppo'] != 'None')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
191,855851453814013952,NaN,NaN,2017-04-22 18:31:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo participating in the #ScienceMa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855851453...,13,10,None,doggo,None,None,puppo


In [35]:
tw[(tw['pupper'] != 'None') & (tw['floofer'] != 'None')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [36]:
tw[(tw['puppo'] != 'None') & (tw['floofer'] != 'None')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [37]:
tw[(tw['pupper'] != 'None') & (tw['puppo'] != 'None')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [10]:
tw.in_reply_to_user_id.nunique()

31

In [59]:
tw[tw['name']=='quite'].text

118     RT @dog_rates: We only rate dogs. This is quit...
169     We only rate dogs. This is quite clearly a smo...
193     Guys, we only rate dogs. This is quite clearly...
2326    This is quite the dog. Gets really excited whe...
Name: text, dtype: object

In [9]:
im

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [120]:
im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [38]:
sum(im.jpg_url.duplicated())

66

In [ ]:
im[im.jpg_url.duplicated()]

In [28]:
im.p1_conf.nunique()

2006

In [ ]:
im.jpg_url.value_counts()

In [124]:
im.tweet_id.nunique()

2075

In [30]:
im.p1.nunique(), im.p1_conf.sort_values(), im.p1_dog.value_counts()

(378, 38      0.044333
 136     0.055379
 1093    0.059033
 1370    0.063152
 246     0.070076
 250     0.071124
 145     0.071536
 680     0.072885
 701     0.081101
 1831    0.082489
 18      0.086502
 109     0.088530
 568     0.088540
 301     0.089165
 1627    0.090341
 1503    0.090341
 954     0.090508
 277     0.096063
 2074    0.097049
 664     0.097232
 789     0.097500
 515     0.098283
 1664    0.099804
 731     0.100499
 1723    0.100896
 1037    0.105171
 247     0.107317
 876     0.107948
 866     0.110587
 1245    0.111493
           ...   
 76      0.999091
 1988    0.999120
 2045    0.999201
 863     0.999223
 1872    0.999281
 1548    0.999306
 95      0.999335
 611     0.999365
 1711    0.999403
 512     0.999484
 168     0.999614
 107     0.999647
 1796    0.999715
 1455    0.999823
 1687    0.999828
 1725    0.999833
 331     0.999834
 1014    0.999837
 594     0.999846
 475     0.999876
 865     0.999885
 45      0.999888
 1447    0.999916
 242     0.999924
 230 

In [134]:
im.p2.nunique(), im.p2.value_counts(), im.p2_dog.value_counts()

(405, Labrador_retriever                104
 golden_retriever                   92
 Cardigan                           73
 Chihuahua                          44
 Pomeranian                         42
 Chesapeake_Bay_retriever           41
 French_bulldog                     41
 toy_poodle                         37
 cocker_spaniel                     34
 miniature_poodle                   33
 Siberian_husky                     33
 beagle                             28
 Eskimo_dog                         27
 collie                             27
 Pembroke                           27
 kuvasz                             26
 Italian_greyhound                  22
 American_Staffordshire_terrier     21
 Pekinese                           21
 chow                               20
 malinois                           20
 miniature_pinscher                 20
 Samoyed                            20
 toy_terrier                        20
 Norwegian_elkhound                 19
 Boston_bull        

In [133]:
im.p3.nunique(), im.p3.value_counts(), im.p3_dog.value_counts()

(408, Labrador_retriever                79
 Chihuahua                         58
 golden_retriever                  48
 Eskimo_dog                        38
 kelpie                            35
 kuvasz                            34
 Staffordshire_bullterrier         32
 chow                              32
 beagle                            31
 cocker_spaniel                    31
 Pomeranian                        29
 toy_poodle                        29
 Pekinese                          29
 Chesapeake_Bay_retriever          27
 Pembroke                          27
 Great_Pyrenees                    27
 French_bulldog                    26
 malamute                          26
 American_Staffordshire_terrier    24
 pug                               23
 Cardigan                          23
 basenji                           21
 bull_mastiff                      20
 toy_terrier                       20
 Siberian_husky                    19
 Boston_bull                       17
 Shetla

In [18]:
tw_ext.head()

,favorite_count,tweet_id,retweet_count,truncated
0,38136,892420643555336193,8345,False
1,32715,892177421306343426,6166,False
2,24629,891815181378084864,4079,False
3,41487,891689557279858688,8488,False
4,39650,891327558926688256,9179,False


In [19]:
tw_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 4 columns):
favorite_count    2342 non-null object
tweet_id          2342 non-null object
retweet_count     2342 non-null object
truncated         2342 non-null object
dtypes: object(4)
memory usage: 73.3+ KB


In [27]:
tw_ext.describe()

,favorite_count,tweet_id,retweet_count,truncated
count,2342,2342,2342,2342
unique,1997,2342,1743,1
top,0,667495797102141441,128,False
freq,168,1,5,2342


In [20]:
tw_ext.truncated.unique()

array([False], dtype=object)

In [25]:
tw_ext.retweet_count.sort_values()

283         0
266         2
333         2
1281        2
110         3
29          4
1066        6
54          7
63         10
417        10
2325       12
2284       14
1509       17
180        17
173        18
2305       20
206        23
2175       25
1223       25
182        28
399        28
600        29
2210       32
953        32
2246       33
872        34
337        35
2245       35
2323       35
277        36
        ...  
437     20708
438     20708
925     22643
301     22643
440     22838
519     22838
1063    22896
877     22896
643     23169
1611    23169
149     23671
646     24464
112     26167
613     26272
449     29743
294     29743
163     30895
159     31506
857     31506
1816    32753
1752    33892
130     35536
527     38350
439     40153
436     40153
65      43551
403     47850
524     61137
1065    62148
1025    83897
Name: retweet_count, Length: 2342, dtype: object

In [26]:
tw_ext.favorite_count.sort_values()

298          0
935          0
443          0
440          0
437          0
436          0
759          0
762          0
428          0
153          0
154          0
898          0
425          0
424          0
421          0
159          0
767          0
415          0
412          0
165          0
410          0
405          0
998          0
401          0
756          0
782          0
929          0
141          0
923          0
709          0
         ...  
646      48513
55       48659
53       49814
1099     51559
131      52586
857      52647
613      53529
385      55258
449      55365
356      56067
7        64346
119      65329
22       67960
347      70713
41       71873
149      75039
37       76233
1816     77622
76       78600
527      80569
112      82118
1752     82497
163      90930
439      92136
65      104250
1065    121671
130     122217
524     127364
403     140744
1025    164649
Name: favorite_count, Length: 2342, dtype: object

In [83]:
tw_ext[tw_ext.tweet_id.duplicated()]

,retweet_count,favorite_count,truncated,tweet_id


###  <font color='blue'> Quality </font>

Twitter archive data: tw table

       1. Missing values in retweeted_status_user_id, retweeted_status_timestamp,in_reply_to_status_id and in_reply_to_user_id  variables 
        Note: We only want original ratings (no retweets) that have images. Then, we will not need 
        retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp anymore.).
        Missing values is a quality issue, while remove columns is tidness issue. 
        However, I will deal with this point in quality section.

       2. Erroneous datatypes:
            - in_reply_to_status_id         is a float not an integer
            - in_reply_to_user_id           is a float not an integer
            - retweeted_status_id           is a float not an integer
            - retweeted_status_user_id      is a float not an integer
            - retweeted_status_timestamp    is a string not a datetime
            - timestamp                     is a string not a datetime
                
       3. rating_denominator variable have values <10. text tweets show errors when extrating these numbers.      
       4. There are four categories for source variable. there is some irrelevant text: html code.   
       5. one character or invalid name.
       6. Same dog with two stage: some values in doggo, floofer, pupper and puppo columns need to be revised. 
       7. Values in doggo, floofer, pupper and puppo columns are bad represented.
       8. Null values are represented in the table sometimes by None and sometimes by NaN.
      
    

Image predictions data: im table

       9. There are duplicate records.


Twitter extension data : tw_ext

      - type of tweet_id, favorite_count and retweet_count is string not integer.
      - truncated column has one value: False.





### <span style="color:blue">Tidiness</span>  

Twitter archive:  tw table

    1. doggo, floofer, pupper and puppo column should be merged into on column: stage
    2. Rating values should be one column 

Image predictions: im tabe

    3. p1, p2 and p3 columns should be merged into two columns: prediction algorithms and breed prediction.
    4. p1_conf, p2_conf and p3_conf columns should be merged into one column: prediction coefficent.
    5. p1_dog, p2_dog and p3_dog should be merged into one column: is_a_Dog
    6. This table should be concatenated with tw table.
 
Twitter exentension data: tw_ext

    7. This table should be concatenated with tw table.

In [ ]:
tw_ext.retweet_count.value_counts()
tw_ext.retweet_count.value_counts()

## Clean

In [5]:
# Create table copies
tw_clean = tw.copy()
im_clean = im.copy()
tw_ext_clean = tw_ext.copy()

### <span style="color:blue">A. Quality</span> 

### 1. Twitter archive: tw table

#### Missing Data
    Missing values in retweeted_status_user_id, retweeted_status_timestamp,  in_reply_to_status_id and in_reply_to_user_id variables. 
    It is a tidness issues, but I see that it must first be treated.

##### Define
We only want original ratings (no retweets) that have images. 

        Remove rows when retweeted_status_id is not null.
        Remove rows whenin_reply_to_status_id is not null.
        Remove these columns and their related columns.

##### Code

In [6]:
#  Remove rows when retweeted_status_id and in_reply_to_status_id are not null. retweeted_status_id not null row will removed automatically.
tw_clean = tw_clean[tw_clean['retweeted_status_id'].isnull()]
tw_clean = tw_clean[tw_clean['in_reply_to_status_id'].isnull()]

In [7]:
#  Remove these columns and their related columns.
tw_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis = 1, inplace = True)
tw_clean.drop(['in_reply_to_status_id','in_reply_to_user_id'], axis = 1, inplace = True)

##### Test

In [8]:
tw_clean.head(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


#### Erroneous datatypes
timestamp is a string not a datetime

##### Define
Transform timestamp type to datetime using to_datetime() method

##### Code

In [9]:
tw_clean['timestamp'] = pd.to_datetime(tw_clean['timestamp'])

##### Test

In [10]:
tw_clean['timestamp'].head(1)

0   2017-08-01 16:23:56
Name: timestamp, dtype: datetime64[ns]

###  rating_denominator variable has values < 10 values
text tweets show errors when extrating these numbers. 

##### Define
Display text tweets and correct values related to this variable.

In [11]:
tw_clean[tw_clean['rating_denominator'] <10]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
516,810984652412424192,2016-12-19 23:06:23,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
2335,666287406224695296,2015-11-16 16:11:11,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,https://twitter.com/dog_rates/status/666287406...,1,2,an,None,None,None,None


In [12]:
tw_clean[tw_clean['rating_denominator'] <10].text[516]  

'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx'

In [13]:
tw_clean[tw_clean['rating_denominator'] <10].text[2335] 

'This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv'

##### Code

In [14]:
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null datetime64[ns]
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 213.0+ KB


In [15]:
tw_clean.loc[tw_clean.tweet_id == 666287406224695296 ,['rating_numerator']] = 9
tw_clean.loc[tw_clean.tweet_id == 666287406224695296 ,['rating_denominator']] = 10

In [16]:
tw_clean.loc[tw_clean.tweet_id == 810984652412424192 ,['rating_numerator']] = np.nan
tw_clean.loc[tw_clean.tweet_id == 810984652412424192 ,['rating_denominator']] = np.nan

##### Test

In [17]:
tw_clean[tw_clean['tweet_id'] == 666287406224695296]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2335,666287406224695296,2015-11-16 16:11:11,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,https://twitter.com/dog_rates/status/666287406...,9.0,10.0,an,None,None,None,None


In [18]:
tw_clean.loc[tw_clean.tweet_id == 810984652412424192 ]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
516,810984652412424192,2016-12-19 23:06:23,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,"https://www.gofundme.com/sams-smile,https://tw...",NaN,NaN,Sam,None,None,None,None


#### Source variable
There are four categories for source variable. Irrelevant text observed: html code.


##### Define
Remove htmal code and leave only utility used to post the tweet. 

##### Code

In [19]:
# see reference [1]
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [20]:
# aplly remove_html_tags function to source column
tw_clean.source = tw_clean.source.apply(remove_html_tags)

##### Test

In [21]:
tw_clean.source.unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'Vine - Make a Scene',
       'TweetDeck'], dtype=object)

#### Same dog with two different stages
Some values in doggo, floofer, pupper and puppo columns need to be revised

##### Define

See on the tweet text and modify stage variable values if it's possible.

##### Code

    There is one observation when doggo and floofer are not null at the same time.

In [22]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw['floofer'] != 'None')]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,2017-04-17 16:34:26,Twitter for iPhone,"At first I thought this was a shy doggo, but i...",https://twitter.com/dog_rates/status/854010172...,11.0,10.0,None,doggo,floofer,None,None


In [23]:
# show tweet test
tw_clean[tw_clean['tweet_id']==854010172552949760].text

200    At first I thought this was a shy doggo, but i...
Name: text, dtype: object

    Tweet text is not complete. Go to original tweet using (expanded_urls). Complete text was: "At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs". As we can see the correct stage is Floofer.

In [24]:
# assign None to doggo for this tweet.
tw_clean.loc[tw_clean.tweet_id == 854010172552949760 ,['doggo']] = 'None'

    There is one observation when doggo and puppo are not null at the same time.

In [25]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['puppo'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
191,855851453814013952,2017-04-22 18:31:02,Twitter for iPhone,Here's a puppo participating in the #ScienceMa...,https://twitter.com/dog_rates/status/855851453...,13.0,10.0,None,doggo,None,None,puppo


    Text of the above tweet was: "Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for". Here is a puppo behaves like a doggo.

In [26]:
# assign None to doggo for this tweet.
tw_clean.loc[tw_clean.tweet_id == 855851453814013952 ,['doggo']] = 'None'

    There are 9 observations when doggo and pupper are not null at the same time.

In [27]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,2017-01-07 16:59:28,Twitter for iPhone,"This is Dido. She's playing the lead role in ""...",https://twitter.com/dog_rates/status/817777686...,13.0,10.0,Dido,doggo,None,pupper,None
531,808106460588765185,2016-12-12 00:29:28,Twitter for iPhone,Here we have Burke (pupper) and Dexter (doggo)...,https://twitter.com/dog_rates/status/808106460...,12.0,10.0,None,doggo,None,pupper,None
575,801115127852503040,2016-11-22 17:28:25,Twitter for iPhone,This is Bones. He's being haunted by another d...,https://twitter.com/dog_rates/status/801115127...,12.0,10.0,Bones,doggo,None,pupper,None
705,785639753186217984,2016-10-11 00:34:48,Twitter for iPhone,This is Pinot. He's a sophisticated doggo. You...,https://twitter.com/dog_rates/status/785639753...,10.0,10.0,Pinot,doggo,None,pupper,None
733,781308096455073793,2016-09-29 01:42:20,Vine - Make a Scene,"Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",https://vine.co/v/5rgu2Law2ut,12.0,10.0,None,doggo,None,pupper,None
889,759793422261743616,2016-07-31 16:50:42,Twitter for iPhone,"Meet Maggie &amp; Lila. Maggie is the doggo, L...",https://twitter.com/dog_rates/status/759793422...,12.0,10.0,Maggie,doggo,None,pupper,None
956,751583847268179968,2016-07-09 01:08:47,Twitter for iPhone,Please stop sending it pictures that don't eve...,https://twitter.com/dog_rates/status/751583847...,5.0,10.0,None,doggo,None,pupper,None
1063,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
1113,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


    Modifications were made by returning to the original text.

In [28]:
# modify tweet_id: 817777686764523521
# tweet text: "This is Dido. She's playing the lead role in 
#"Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple." 13/10 (IG: didodoggo)"
tw_clean.loc[tw_clean.tweet_id == 817777686764523521 ,['doggo']] = 'None'

        In the tweet below (tweet_id = 808106460588765185), He rates are two dogs, pupper and doggo. We can divide this observation into 2 different observations. 

In [29]:
# Replicate tweet_id: 808106460588765185
df = tw_clean.loc[tw_clean.tweet_id == 808106460588765185 ]
tw_clean = tw_clean.append(df, ignore_index=True)

In [30]:
# refresh table to get the new index of the replicate row
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
416,808106460588765185,2016-12-12 00:29:28,Twitter for iPhone,Here we have Burke (pupper) and Dexter (doggo)...,https://twitter.com/dog_rates/status/808106460...,12.0,10.0,None,doggo,None,pupper,None
446,801115127852503040,2016-11-22 17:28:25,Twitter for iPhone,This is Bones. He's being haunted by another d...,https://twitter.com/dog_rates/status/801115127...,12.0,10.0,Bones,doggo,None,pupper,None
536,785639753186217984,2016-10-11 00:34:48,Twitter for iPhone,This is Pinot. He's a sophisticated doggo. You...,https://twitter.com/dog_rates/status/785639753...,10.0,10.0,Pinot,doggo,None,pupper,None
562,781308096455073793,2016-09-29 01:42:20,Vine - Make a Scene,"Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",https://vine.co/v/5rgu2Law2ut,12.0,10.0,None,doggo,None,pupper,None
689,759793422261743616,2016-07-31 16:50:42,Twitter for iPhone,"Meet Maggie &amp; Lila. Maggie is the doggo, L...",https://twitter.com/dog_rates/status/759793422...,12.0,10.0,Maggie,doggo,None,pupper,None
748,751583847268179968,2016-07-09 01:08:47,Twitter for iPhone,Please stop sending it pictures that don't eve...,https://twitter.com/dog_rates/status/751583847...,5.0,10.0,None,doggo,None,pupper,None
848,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None
2097,808106460588765185,2016-12-12 00:29:28,Twitter for iPhone,Here we have Burke (pupper) and Dexter (doggo)...,https://twitter.com/dog_rates/status/808106460...,12.0,10.0,None,doggo,None,pupper,None


In [31]:
# Modify doggo in one row and pupper in the other row
tw_clean.loc[2097, 'doggo'] = 'None'
tw_clean.loc[416, 'pupper'] = 'None'

In [32]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
446,801115127852503040,2016-11-22 17:28:25,Twitter for iPhone,This is Bones. He's being haunted by another d...,https://twitter.com/dog_rates/status/801115127...,12.0,10.0,Bones,doggo,None,pupper,None
536,785639753186217984,2016-10-11 00:34:48,Twitter for iPhone,This is Pinot. He's a sophisticated doggo. You...,https://twitter.com/dog_rates/status/785639753...,10.0,10.0,Pinot,doggo,None,pupper,None
562,781308096455073793,2016-09-29 01:42:20,Vine - Make a Scene,"Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",https://vine.co/v/5rgu2Law2ut,12.0,10.0,None,doggo,None,pupper,None
689,759793422261743616,2016-07-31 16:50:42,Twitter for iPhone,"Meet Maggie &amp; Lila. Maggie is the doggo, L...",https://twitter.com/dog_rates/status/759793422...,12.0,10.0,Maggie,doggo,None,pupper,None
748,751583847268179968,2016-07-09 01:08:47,Twitter for iPhone,Please stop sending it pictures that don't eve...,https://twitter.com/dog_rates/status/751583847...,5.0,10.0,None,doggo,None,pupper,None
848,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


In [33]:
# tweet text "This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths 
# pupper everything's fine"
tw_clean.loc[tw_clean.tweet_id == 801115127852503040 ,['doggo']] = 'None'

# tweet text "This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. 
# Still 10/10 would pet cautiously"
tw_clean.loc[tw_clean.tweet_id == 785639753186217984 ,['pupper']] = 'None'

In [34]:
# Text tweetPupper butt 1, Doggo 0. Both 12/10

# Replicate tweet_id: 781308096455073793
df = tw_clean.loc[tw_clean.tweet_id == 781308096455073793 ]
tw_clean = tw_clean.append(df, ignore_index=True)

In [35]:
# Modify doggo in one row and pupper in the other row
tw_clean.loc[2098, 'doggo'] = 'None'
tw_clean.loc[562, 'pupper'] = 'None'

In [36]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
689,759793422261743616,2016-07-31 16:50:42,Twitter for iPhone,"Meet Maggie &amp; Lila. Maggie is the doggo, L...",https://twitter.com/dog_rates/status/759793422...,12.0,10.0,Maggie,doggo,None,pupper,None
748,751583847268179968,2016-07-09 01:08:47,Twitter for iPhone,Please stop sending it pictures that don't eve...,https://twitter.com/dog_rates/status/751583847...,5.0,10.0,None,doggo,None,pupper,None
848,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


In [37]:
# tweet text: "Meet Maggie & Lila. Maggie is the doggo, Lila is the pupper. 
#They are sisters. Both 12/10 would pet at the same time"

# Replicate tweet_id: 759793422261743616
df = tw_clean.loc[tw_clean.tweet_id == 759793422261743616 ]
tw_clean = tw_clean.append(df, ignore_index=True)

In [38]:
# Modify doggo in one row and pupper in the other row
tw_clean.loc[689, 'pupper'] = 'None'
# Modify doggo in one row and pupper in the other row
tw_clean.loc[2099, 'doggo'] = 'None'
tw_clean.loc[689, 'name'] = 'Lila'


In [39]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
748,751583847268179968,2016-07-09 01:08:47,Twitter for iPhone,Please stop sending it pictures that don't eve...,https://twitter.com/dog_rates/status/751583847...,5.0,10.0,None,doggo,None,pupper,None
848,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


In [40]:
# tweet text:  Please stop sending it pictures that don't even have a doggo or pupper in them.
# Churlish af. 5/10 neat couch tho

tw_clean.loc[tw_clean.tweet_id == 751583847268179968 ,['doggo','pupper']] = 'None'

In [41]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
848,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


In [42]:
# tweet text: "This is just downright precious af. 12/10 for both pupper and doggo"

# Replicate tweet_id: 741067306818797568
df = tw_clean.loc[tw_clean.tweet_id == 741067306818797568 ]
tw_clean = tw_clean.append(df, ignore_index=True)

In [43]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
848,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None
2100,741067306818797568,2016-06-10 00:39:48,Twitter for iPhone,This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12.0,10.0,just,doggo,None,pupper,None


In [44]:
# Modify doggo in one row and pupper in the other row
tw_clean.loc[2100, 'doggo'] = 'None'
tw_clean.loc[848, 'pupper'] = 'None'

In [45]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


In [46]:
# tweet text: "Like father (doggo), like son (pupper). Both 12/10"

# Replicate tweet_id: 733109485275860992
df = tw_clean.loc[tw_clean.tweet_id == 733109485275860992 ]
tw_clean = tw_clean.append(df, ignore_index=True)
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
897,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None
2101,733109485275860992,2016-05-19 01:38:16,Twitter for iPhone,"Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12.0,10.0,None,doggo,None,pupper,None


In [47]:
# Modify doggo in one row and pupper in the other row
tw_clean.loc[2101, 'doggo'] = 'None'
tw_clean.loc[897, 'pupper'] = 'None'

##### Test

In [48]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['pupper'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [49]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw['floofer'] != 'None')]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [50]:
tw_clean[(tw_clean['doggo'] != 'None') & (tw_clean['puppo'] != 'None')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [51]:
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 12 columns):
tweet_id              2102 non-null int64
timestamp             2102 non-null datetime64[ns]
source                2102 non-null object
text                  2102 non-null object
expanded_urls         2099 non-null object
rating_numerator      2101 non-null float64
rating_denominator    2101 non-null float64
name                  2102 non-null object
doggo                 2102 non-null object
floofer               2102 non-null object
pupper                2102 non-null object
puppo                 2102 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 197.1+ KB


#### One character or invalid name

##### Define
Remove or correct invalid dog names.
Invalid names usually are lowercase.

##### Code

In [52]:
tw_clean.name.sort_values()

808             Abby
821             Abby
732              Ace
1683            Acro
1108           Adele
1684           Aiden
68               Aja
372            Akumi
632               Al
676           Albert
1703          Albert
123            Albus
324            Albus
899          Aldrick
1793       Alejandro
294        Alexander
972     Alexanderson
377              Alf
1383           Alfie
662            Alfie
1982           Alfie
287            Alfie
47              Alfy
162            Alice
1463           Alice
1267           Amber
1114         Ambrose
1825             Amy
1939          Amélie
598           Anakin
            ...     
1750           light
879              mad
657               my
779              not
263              not
1509      officially
989              old
1686             one
784              one
289              one
720              one
141            quite
156            quite
2067           quite
1779           space
21              such
1785         

In [53]:
# There are 105 invalid names.
indices = tw_clean[tw_clean.name.str.islower()].index
tw_clean[tw_clean.name.str.islower()]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
21,887517139158093824,2017-07-19 03:39:09,Twitter for iPhone,I've yet to rate a Venezuelan Hover Wiener. Th...,https://twitter.com/dog_rates/status/887517139...,14.0,10.0,such,None,None,None,None
51,881536004380872706,2017-07-02 15:32:16,Twitter for iPhone,Here is a pupper approaching maximum borkdrive...,https://twitter.com/dog_rates/status/881536004...,14.0,10.0,a,None,None,pupper,None
141,859196978902773760,2017-05-02 00:04:57,Twitter for iPhone,We only rate dogs. This is quite clearly a smo...,https://twitter.com/dog_rates/status/859196978...,12.0,10.0,quite,None,None,None,None
156,855459453768019968,2017-04-21 16:33:22,Twitter for iPhone,"Guys, we only rate dogs. This is quite clearly...",https://twitter.com/dog_rates/status/855459453...,12.0,10.0,quite,None,None,None,None
263,832645525019123713,2017-02-17 17:38:57,Twitter Web Client,There's going to be a dog terminal at JFK Airp...,http://us.blastingnews.com/news/2017/02/jfk-an...,10.0,10.0,not,None,None,None,None
289,828650029636317184,2017-02-06 17:02:17,Twitter for iPhone,"Occasionally, we're sent fantastic stories. Th...",https://twitter.com/dog_rates/status/828650029...,14.0,10.0,one,None,None,None,None
424,806219024703037440,2016-12-06 19:29:28,Twitter for iPhone,We only rate dogs. Please stop sending in non-...,https://twitter.com/dog_rates/status/806219024...,11.0,10.0,incredibly,None,None,None,None
494,792913359805018113,2016-10-31 02:17:31,Twitter for iPhone,Here is a perfect example of someone who has t...,https://twitter.com/dog_rates/status/792913359...,13.0,10.0,a,None,None,None,None
616,772581559778025472,2016-09-04 23:46:12,Twitter for iPhone,Guys this is getting so out of hand. We only r...,https://twitter.com/dog_rates/status/772581559...,10.0,10.0,a,None,None,None,None
631,770655142660169732,2016-08-30 16:11:18,Twitter for iPhone,We only rate dogs. Pls stop sending in non-can...,https://twitter.com/dog_rates/status/770655142...,11.0,10.0,very,None,None,None,None


I checked some text of the above tweets. There is no name of the rated dogs.

In [54]:
tw_clean.loc[indices, 'name'] = 'None'

##### Test

In [55]:
tw_clean[tw_clean.name.str.islower()]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


#### Values in doggo, floofer, pupper and puppo columns are bad represented

##### Define

Transform doggo,floofer, pupper and	puppo variables to a dummy variables:
- Remplace doggo,floofer, pupper and puppo **values** in doggo,	floofer, pupper and puppo **columns** by **1**.
- Remplace None **values** in doggo, floofer, pupper and puppo **columns** by **0**. 

##### Code

In [56]:
tw_clean1 = tw_clean.copy()

In [57]:
tw_clean.replace({'doggo': 'None'}, {'doggo': 0}, inplace = True)
tw_clean.replace({'floofer': 'None'}, {'floofer': 0}, inplace = True)
tw_clean.replace({'pupper': 'None'}, {'pupper': 0}, inplace = True)
tw_clean.replace({'puppo': 'None'}, {'puppo': 0}, inplace = True)
tw_clean.replace(['doggo','floofer','pupper','puppo'], 1, inplace = True)

##### Test

In [58]:
tw_clean.head(10)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13.0,10.0,Phineas,0,0,0,0
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13.0,10.0,Tilly,0,0,0,0
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12.0,10.0,Archie,0,0,0,0
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13.0,10.0,Darla,0,0,0,0
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12.0,10.0,Franklin,0,0,0,0
5,891087950875897856,2017-07-29 00:08:17,Twitter for iPhone,Here we have a majestic great white breaching ...,https://twitter.com/dog_rates/status/891087950...,13.0,10.0,None,0,0,0,0
6,890971913173991426,2017-07-28 16:27:12,Twitter for iPhone,Meet Jax. He enjoys ice cream so much he gets ...,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13.0,10.0,Jax,0,0,0,0
7,890729181411237888,2017-07-28 00:22:40,Twitter for iPhone,When you watch your owner call another dog a g...,https://twitter.com/dog_rates/status/890729181...,13.0,10.0,None,0,0,0,0
8,890609185150312448,2017-07-27 16:25:51,Twitter for iPhone,This is Zoey. She doesn't want to be one of th...,https://twitter.com/dog_rates/status/890609185...,13.0,10.0,Zoey,0,0,0,0
9,890240255349198849,2017-07-26 15:59:51,Twitter for iPhone,This is Cassie. She is a college pup. Studying...,https://twitter.com/dog_rates/status/890240255...,14.0,10.0,Cassie,1,0,0,0


In [59]:
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 12 columns):
tweet_id              2102 non-null int64
timestamp             2102 non-null datetime64[ns]
source                2102 non-null object
text                  2102 non-null object
expanded_urls         2099 non-null object
rating_numerator      2101 non-null float64
rating_denominator    2101 non-null float64
name                  2102 non-null object
doggo                 2102 non-null int64
floofer               2102 non-null int64
pupper                2102 non-null int64
puppo                 2102 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 197.1+ KB


In [60]:
tw_clean.doggo.value_counts()

0    2024
1      78
Name: doggo, dtype: int64

#### Null values are represented differently
in the tw table, sometimes by None and sometimes by NaN.

##### Define

NaNs represent missing values in rating columns. Nones represent missing vales in name and stage columns. Can I say that is a concistency issue? A debate on the defference between them can be found ine reference [2]. For the moment, I leave them as they are. 

### 2. Image prediction: im table

#### There are duplicate records. 

##### Define

Remove duplicate rows

##### Code

In [61]:
im_clean.drop_duplicates(inplace=True)

##### Test

In [62]:
sum(im_clean.duplicated())

0

### 3. Twitter archive Extension : tw_ext table

#### Erroneous datatypes
Type of tweet_id, favorite_count and retweet_count is string not integer.

##### Define

Modify type of the varibale using astype() method

##### Code

In [63]:
tw_ext_clean.tweet_id       = tw_ext_clean.tweet_id.astype(int)
tw_ext_clean.favorite_count = tw_ext_clean.favorite_count.astype(int)
tw_ext_clean.retweet_count  = tw_ext_clean.retweet_count.astype(int)

##### Test

In [64]:
tw_ext_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 4 columns):
retweet_count     2342 non-null int64
truncated         2342 non-null object
favorite_count    2342 non-null int64
tweet_id          2342 non-null int64
dtypes: int64(3), object(1)
memory usage: 73.3+ KB


#### 'truncated' column has one value: False

##### Define

Remove 'truncated' column

##### Code

In [65]:
# truncated column contains one value: False. I will remove it
tw_ext_clean.drop(['truncated'], axis = 1, inplace = True)

##### Test

In [66]:
tw_ext_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
retweet_count     2342 non-null int64
favorite_count    2342 non-null int64
tweet_id          2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


### <span style="color:blue">B. Tidiness</span> 

### Twitter archive: tw table

#### doggo, floofer, pupper and puppo columns should be merged into one column: stage

##### Define

Melt doggo, floofer, pupper and puppo columns to stage column.
Reconstruct a categorical variable from dummies.

##### Code

In [67]:
# Reconstruct a categorical variable from dummies
tw_clean['stage'] = tw_clean[['doggo','floofer','pupper','puppo']].idxmax(axis=1)
#  Create mask for 0 value in doggo floofer pupper puppo
mask = (tw_clean['doggo'] == 0) & (tw_clean['floofer'] == 0) & (tw_clean['pupper'] == 0) & (tw_clean['puppo'] == 0) 
# set satge value to None if all doggo floofer pupper puppo values are 0
tw_clean.loc[mask, 'stage'] = 'None'
# remove  doggo floofer pupper puppo columns
tw_clean.drop(['doggo','floofer','pupper','puppo'], axis = 1, inplace = True)

##### Test

In [68]:
tw_clean.head(10)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,stage
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13.0,10.0,Phineas,None
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13.0,10.0,Tilly,None
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12.0,10.0,Archie,None
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13.0,10.0,Darla,None
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12.0,10.0,Franklin,None
5,891087950875897856,2017-07-29 00:08:17,Twitter for iPhone,Here we have a majestic great white breaching ...,https://twitter.com/dog_rates/status/891087950...,13.0,10.0,None,None
6,890971913173991426,2017-07-28 16:27:12,Twitter for iPhone,Meet Jax. He enjoys ice cream so much he gets ...,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13.0,10.0,Jax,None
7,890729181411237888,2017-07-28 00:22:40,Twitter for iPhone,When you watch your owner call another dog a g...,https://twitter.com/dog_rates/status/890729181...,13.0,10.0,None,None
8,890609185150312448,2017-07-27 16:25:51,Twitter for iPhone,This is Zoey. She doesn't want to be one of th...,https://twitter.com/dog_rates/status/890609185...,13.0,10.0,Zoey,None
9,890240255349198849,2017-07-26 15:59:51,Twitter for iPhone,This is Cassie. She is a college pup. Studying...,https://twitter.com/dog_rates/status/890240255...,14.0,10.0,Cassie,doggo


In [69]:
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 9 columns):
tweet_id              2102 non-null int64
timestamp             2102 non-null datetime64[ns]
source                2102 non-null object
text                  2102 non-null object
expanded_urls         2099 non-null object
rating_numerator      2101 non-null float64
rating_denominator    2101 non-null float64
name                  2102 non-null object
stage                 2102 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 147.9+ KB


#### Rating values should be one column 

##### Define

Add a nex column 'Rating' and assign it the result of (rating_numerator/rating_denominator)

##### Code

In [70]:
tw_clean['Rating'] = tw_clean.rating_numerator/tw_clean.rating_denominator
tw_clean = tw_clean.drop(['rating_numerator','rating_denominator'], axis =1 )

##### Test

In [71]:
tw_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,name,stage,Rating
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,None,1.3
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,None,1.3
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,None,1.2
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,None,1.3
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,None,1.2


In [72]:
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 8 columns):
tweet_id         2102 non-null int64
timestamp        2102 non-null datetime64[ns]
source           2102 non-null object
text             2102 non-null object
expanded_urls    2099 non-null object
name             2102 non-null object
stage            2102 non-null object
Rating           2101 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 131.5+ KB


### Image prediction : im table

####  Columns should be merged  
    2. p1, p2 and p3 columns should be merged into two columns: prediction algorithms and image prediction.
    3. p1_conf, p2_conf and p3_conf columns should be merged into one column: prediction coefficent.
    4. p1_dog, p2_dog and p3_dog should be merged into one column: is_a_Dog

##### Define

    Melt p1, p2 and p3 columns into two columns: 'pred_algo' and 'imag_pred'
    Melt p1_conf, p2_conf and p3_conf columns to one column: pred_coef.
    Melt p1_dog, p2_dog and p3_dog to one column: is_a_dog

##### Code

In [73]:
im_clean.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [74]:
im_clean = im_clean.set_index(['tweet_id','jpg_url','img_num']).sort_index(axis=1)

# extract primary columns
prim = im_clean.filter(like='1')
prim.columns = ['img_pred', 'pred_coef', 'is_a_dog']
# extract secondary columns 
sec = im_clean.filter(like='2')
sec.columns = ['img_pred', 'pred_coef', 'is_a_dog']
# extract tertiary columns 
rd = im_clean.filter(like='3')
rd.columns = ['img_pred', 'pred_coef', 'is_a_dog']

# concatenation + housekeeping
im_clean = (pd.concat([prim, sec, rd], keys=['algo1', 'algo2', 'algo3'])
       .swaplevel(0,1)
       .swaplevel(1,3)
       .swaplevel(1,2)
       .rename_axis(['tweet_id','jpg_url','img_num', 'pred_algo'])
       .reset_index()
)

##### Test

In [75]:
im_clean.head(3)

,tweet_id,jpg_url,img_num,pred_algo,img_pred,pred_coef,is_a_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo1,Welsh_springer_spaniel,0.465074,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,algo1,redbone,0.506826,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,algo1,German_shepherd,0.596461,True


In [76]:
im_clean.query('tweet_id == 666020888022790149')

,tweet_id,jpg_url,img_num,pred_algo,img_pred,pred_coef,is_a_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo1,Welsh_springer_spaniel,0.465074,True
2075,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo2,collie,0.156665,True
4150,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo3,Shetland_sheepdog,0.061428,True


#### The 3 tables should be concatenated.

##### Define

Mege tw_clean, tw_ext_clean and im_clean tables on tweet_id. tweet without images will be discarded.

##### Code

In [77]:
rate_dogs = tw_clean.merge(tw_ext_clean, on = 'tweet_id')
rate_dogs = rate_dogs.merge(im_clean, on = 'tweet_id')

##### Test

In [78]:
rate_dogs.head(3)

,tweet_id,timestamp,source,text,expanded_urls,name,stage,Rating,retweet_count,favorite_count,jpg_url,img_num,pred_algo,img_pred,pred_coef,is_a_dog
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,None,1.3,8345,38136,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,algo1,orange,0.097049,False
1,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,None,1.3,8345,38136,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,algo2,bagel,0.085851,False
2,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,None,1.3,8345,38136,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,algo3,banana,0.076110,False


In [79]:
rate_dogs.query('tweet_id == 666020888022790149')

,tweet_id,timestamp,source,text,expanded_urls,name,stage,Rating,retweet_count,favorite_count,jpg_url,img_num,pred_algo,img_pred,pred_coef,is_a_dog
5919,666020888022790149,2015-11-15 22:32:08,Twitter for iPhone,Here we have a Japanese Irish Setter. Lost eye...,https://twitter.com/dog_rates/status/666020888...,None,None,0.8,506,2537,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo1,Welsh_springer_spaniel,0.465074,True
5920,666020888022790149,2015-11-15 22:32:08,Twitter for iPhone,Here we have a Japanese Irish Setter. Lost eye...,https://twitter.com/dog_rates/status/666020888...,None,None,0.8,506,2537,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo2,collie,0.156665,True
5921,666020888022790149,2015-11-15 22:32:08,Twitter for iPhone,Here we have a Japanese Irish Setter. Lost eye...,https://twitter.com/dog_rates/status/666020888...,None,None,0.8,506,2537,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,algo3,Shetland_sheepdog,0.061428,True


# Storing combined and cleaned data

In [80]:
rate_dogs.to_csv('twitter_archive_master.csv', index = False)

# Anayze and Visualize cleaned data

You can find an analysis and visualization of the cleaned data in act_report.html file.

# Reference

   * [[1]](https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44)How to strip html tags from a string in Python.
   * [[2]](https://stackoverflow.com/questions/17534106/what-is-the-difference-between-nan-and-none) What is the difference between NaN and None?
   * [[3]](https://stackoverflow.com/questions/26762100/reconstruct-a-categorical-variable-from-dummies-in-pandas) Reconstruct a categorical variable from dummies in pandas.
   * [[4]](https://stackoverflow.com/questions/50770021/pandas-melt-multiple-groups-into-single-column) Melt multiple groups into single column.

In [2]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'wrangle_act.ipynb'])

0

In [ ]:
!tar chvfz notebook.tar.gz *